# Payoff options vanille (call / put)

Payoff brut et P&L net (BS).


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent, _base.parent.parent.parent):
    candidate = extra / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
plt.style.use('seaborn-v0_8-darkgrid')

from pricing import fetch_spy_history, view_vanilla_call, view_vanilla_put
close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])
strike_ref = spot_ref
sT_ref = spot_ref


## Payoff interactif
Choix call/put, S0, K, sigma, r, T. Prix via BS (affiché ci-dessous).


In [ ]:
sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, description='Sigma')
r_slider = widgets.FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.005, description='r')
T_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='T')

option_dd = widgets.Dropdown(options=[('Call','call'),('Put','put')], value='call', description='Type')
spot_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S0')
strike_slider = widgets.FloatSlider(value=strike_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K')
st_slider = widgets.FloatSlider(value=sT_ref, min=spot_ref*0.3, max=spot_ref*2.0, step=1.0, description='S_T')
output = widgets.Output()
VIEW = {'call': view_vanilla_call, 'put': view_vanilla_put}

def _update(change=None):
    with output:
        output.clear_output()
        kind = option_dd.value
        s0 = spot_slider.value
        k = strike_slider.value
        sT_val = st_slider.value
        view_dyn = VIEW[kind](s0, k, r=r_slider.value, q=0.0, sigma=sigma_slider.value, T=T_slider.value)
        premium = float(view_dyn.get('premium', 0.0))
        s_grid = view_dyn['s_grid']
        payoff_grid = view_dyn['payoff']
        pnl_grid = view_dyn['pnl']
        payoff_s0 = float(np.interp(s0, s_grid, payoff_grid))
        payoff_st = float(np.interp(sT_val, s_grid, payoff_grid))
        pnl_s0 = payoff_s0 - premium
        fig, ax = plt.subplots(figsize=(7,4))
        ax.plot(s_grid, payoff_grid, label='Payoff')
        ax.plot(s_grid, pnl_grid, label='P&L net', color='darkorange')
        ax.axvline(k, color='gray', linestyle='--', label=f'K = {k:.2f}')
        ax.axvline(s0, color='crimson', linestyle='-.', label=f'S0 = {s0:.2f}')
        ax.axvline(sT_val, color='purple', linestyle=':', label=f'S_T = {sT_val:.2f}')
        ax.axhline(0, color='black', linewidth=0.8)
        ax.legend(loc='best')
        ax.set_xlabel('Spot')
        ax.set_ylabel('Payoff / P&L')
        ax.set_title(f'Vanilla {kind}')
        plt.show()
        plt.close(fig)
        msg = (
            f"Prix option = {premium:.4f}\n\n"
            f"- Payoff @ S0 = {payoff_s0:.4f}\n"
            f"- Payoff @ S_T = {payoff_st:.4f}\n"
            f"- P&L net (S0) = {pnl_s0:.4f}"
        )
        display(Markdown(msg))

for sl in (option_dd, spot_slider, strike_slider, sigma_slider, r_slider, T_slider, st_slider):
    sl.observe(_update, names='value')
_update()
display(widgets.VBox([option_dd, spot_slider, strike_slider, st_slider, sigma_slider, r_slider, T_slider, output]))

